In [1]:
def Our_model():
    from keras.layers import Input, Embedding, Flatten, Dot,Dense,Dropout
    from keras.models import Model
    from keras.layers import Concatenate
    # For each sample we input the integer identifiers
    # of a single user and a single item
    user_id_input = Input(shape=[1],name='user')
    item_id_input = Input(shape=[1], name='item')
    meta_input = Input(shape=[5], name='meta_item')
    
    embedding_size = 35
    user_embedding = Embedding(output_dim=embedding_size, input_dim=nb_users + 1,
                               input_length=1, name='user_embedding')(user_id_input)

    item_embedding = Embedding(output_dim=embedding_size, input_dim=nb_items + 1,
                               input_length=1, name='item_embedding')(item_id_input)

    # reshape from shape: (batch_size, input_length, embedding_size)
    # to shape: (batch_size, input_length * embedding_size) which is
    # equal to shape: (batch_size, embedding_size)
    user_vecs = Flatten()(user_embedding)
    item_vecs = Flatten()(item_embedding)
    
    y = Concatenate()([user_vecs, item_vecs, meta_input])
    x = Dense(64, activation='relu')(y)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    y = Dense(1)(x)
    model = Model(inputs=[user_id_input, item_id_input, meta_input], outputs=y)
    model.compile(optimizer="adam", loss="MAE")
    return model

In [2]:
%%time
import requests
from time import sleep 
import numpy as np
from keras.callbacks import EarlyStopping
user_id= '0SIMZCTG7KZACJPQB7FU'
base_url='http://35.180.254.42'
url_reset=base_url+'/reset'
url_predict=base_url+'/predict'
params={'user_id':user_id}
data= requests.get(url_reset, params=params).json()
nb_users=data['nb_users']
nb_items=data['nb_items']
rating_history=data['rating_history']
item_history=data['item_history']
variables_history=data['variables_history']
next_user=data['next_user']
next_item=data['next_item']
next_variables=data["next_variables"]
nb_samples=1000
mse,mae=0,0
user_history=data['user_history']
model=Our_model()
early_stopping = EarlyStopping(monitor='val_loss', patience=4)
history = model.fit([user_history, item_history,np.array(variables_history)], rating_history,
                    batch_size=64, epochs=20, validation_split=0.1,
                    shuffle=True, callbacks=[early_stopping])
for i in range(nb_samples):
    sleep(0.05)
    prediction = model.predict([np.array([next_user]), np.array([next_item]),np.array(next_variables).reshape(1,5)] )
    params['predicted_score']=prediction
    d=requests.get(url_predict, params=params).json()
    rating=d['rating']
    #print(f'user:{next_user},item:{next_item},rating:{rating},prediction:{prediction}')
    next_user=d['next_user']
    next_item=d['next_item']
    next_variables=d['next_variables']
    mse+=(rating-prediction)**2
    mae+=abs(rating-prediction)
print('mse: ', mse/nb_samples)
print('mae: ', mae/nb_samples)

C:\Users\RAMZI\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Train on 1800 samples, validate on 200 samples
Epoch 1/20
1800/1800 [==============================] - 0s 189us/step - loss: 2.2743 - val_loss: 1.5256
Epoch 2/20
1800/1800 [==============================] - 0s 22us/step - loss: 1.4552 - val_loss: 1.2910
Epoch 3/20
1800/1800 [==============================] - 0s 23us/step - loss: 1.3542 - val_loss: 1.2119
Epoch 4/20
1800/1800 [==============================] - 0s 24us/step - loss: 1.2057 - val_loss: 1.0683
Epoch 5/20
1800/1800 [==============================] - 0s 22us/step - loss: 1.0345 - val_loss: 0.9047
Epoch 6/20
1800/1800 [==============================] - 0s 22us/step - loss: 0.9166 - val_loss: 0.8264
Epoch 7/20
1800/1800 [==============================] - 0s 22us/step - loss: 0.8233 - val_loss: 0.8069
Epoch 8/20
1800/1800 [==============================] - 0s 27us/step - loss: 0.7968 - val_loss: 0.8188
Epoch 9/20
1800/1800 [==============================] - 0s 24us/step - loss: 0.7512 - val_loss: 0.8226
Epoch 10/20
1800/1800 [==